In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np

In [10]:
# cur_task = "ProbabilisticReasoning"
cur_task = "HorizonTask"
res_dir = "Experiments/" + cur_task + "/"
score_csv = res_dir + "scores_data.csv"
# model = "gemma-2-9b-it"
model = "Llama-3.1-8B-Instruct"

In [11]:
results = {}


with open(score_csv, "r") as f:
    lines = f.readlines()
    columns = lines[0].strip().split(",")
    lines = lines[1:]
    lines = [line.strip().split(",") for line in lines]
    for line in lines:
        if "str_" not in line[0]:
            continue
        if model not in line[0]:
            continue
        steer_str, llm = line[0].split("+")
        if steer_str == "str_none":
            str_type = "none"
            str_str = 0.0
        else:
            str_type, str_str = steer_str.split("_")[1:]
        if str_type not in results:
            results[str_type] = {}
        for i in range(1, len(columns)):
            if columns[i] not in results[str_type]:
                results[str_type][columns[i]] = []
            results[str_type][columns[i]].append(line[i])
        

In [12]:
results['random 0']

{'run': ['0'],
 'performance_score1': ['53.559477124183005'],
 'performance_score1_name': ['rewards mean'],
 'behaviour_score1': ['0.170657811464368'],
 'behaviour_score1_name': ['Directed Exploration'],
 'behaviour_score2': ['0.0082183186993949'],
 'behaviour_score2_name': ['Random Exploration'],
 'behaviour_score1_CI': ['0.7197424574779415'],
 'behaviour_score2_CI': ['0.1006235728044118'],
 'performance_score1_CI': ['1.2796117405130476']}

In [13]:
symptom_names = list(results.keys())
for i in range(10):
    symptom_names.remove("random "+str(i))
symptom_names.remove("none")

In [14]:
symptom_names

['depressed mood', 'risk-aversion', 'manic mood', 'risk-seeking']

In [15]:
import matplotlib.pyplot as plt
from scipy import stats
x = [0.0, 0.25]

n_symptoms = len(symptom_names)
cols = 3
rows = (n_symptoms + 2) // cols
if rows == 0:
    rows = 1

fig, axes = plt.subplots(rows, cols, figsize=(15, 3 * rows))
axes = axes.flatten()
# plt.title('Good Scores on SocialChemistry101')
max_val = None
min_val = None
for idx, symptom in enumerate(symptom_names):
    zero_val = results["none"]['performance_score1'][0]
    vals = [float(zero_val)]
    vals = vals + list(map(float, results[symptom]['performance_score1'][1:]))
    axes[idx].plot(x, vals, marker='o', label=symptom, color='blue')

    cur_max = max(vals)
    cur_min = min(vals)

    if max_val is None:
        max_val = cur_max
    else:
        max_val = max(max_val, cur_max)
    if min_val is None:
        min_val = cur_min
    else:
        min_val = min(min_val, cur_min)

    vals = [float(zero_val)]
    neg_vals = list(map(float, results[symptom]['performance_score1'][:1]))
    neg_vals_ordered = []
    for i in range(1):
        neg_vals_ordered.append(neg_vals[0-i])
    vals = vals + neg_vals_ordered
    axes[idx].plot(x, vals, marker='o', linestyle='--', label=symptom + " Negative", color='red')
    cur_max = max(vals)
    cur_min = min(vals)
    max_val = max(max_val, cur_max)
    min_val = min(min_val, cur_min)
    # axes[idx].plot(x, list(map(float, results[symptom]['performance_score1'])), marker='o')
    random_values = []
    for i in range(10):
        cur_vals = list(map(float, results["random "+str(i)]['performance_score1']))
        random_values.append([float(zero_val)] + cur_vals)
    random_values = np.array(random_values)
    random_mean = np.mean(random_values, axis=0)
    lower_bound = np.mean(random_values, axis=0) - stats.sem(random_values, axis=0)
    sem = stats.sem(random_values)
    ci = stats.t.ppf(0.95, len(random_values) - 1) * sem
    lower_bound = random_mean - ci
    upper_bound = random_mean + ci
    axes[idx].plot(x, random_mean, marker='o', linestyle='--', label='Random Mean', color='orange')

    axes[idx].fill_between(x, lower_bound, upper_bound, color='orange', alpha=0.3, label='Random ±95% CI')
    # random_values = np.mean(random_values, axis=0)

    axes[idx].set_title(symptom, fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Intensity')
    axes[idx].set_ylabel('Score')

for i in range(len(symptom_names)):
    lower_bound = min_val - abs(min_val) * 0.1
    upper_bound = max_val + abs(max_val) * 0.1
    axes[i].set_ylim(lower_bound, upper_bound)

for i in range(n_symptoms, len(axes)):
    fig.delaxes(axes[i])

fig.suptitle("Performance score1 on "+ cur_task, fontsize=20, fontweight='bold')
plt.tight_layout()
plt.savefig("figures/" + model + cur_task + "_" + results["random 0"]['performance_score1_name'][0] +".png", dpi=600)
plt.close()

In [16]:
fig, axes = plt.subplots(rows, cols, figsize=(15, 3 * rows))
axes = axes.flatten()
# plt.title('Good Scores on SocialChemistry101')
max_val = None
min_val = None
for idx, symptom in enumerate(symptom_names):
    zero_val = results["none"]['behaviour_score1'][0]
    vals = [float(zero_val)]
    vals = vals + list(map(float, results[symptom]['behaviour_score1'][1:]))
    axes[idx].plot(x, vals, marker='o', label=symptom, color='blue')

    cur_max = max(vals)
    cur_min = min(vals)

    if max_val is None:
        max_val = cur_max
    else:
        max_val = max(max_val, cur_max)
    if min_val is None:
        min_val = cur_min
    else:
        min_val = min(min_val, cur_min)

    vals = [float(zero_val)]
    neg_vals = list(map(float, results[symptom]['behaviour_score1'][:1]))
    neg_vals_ordered = []
    for i in range(1):
        neg_vals_ordered.append(neg_vals[0-i])
    vals = vals + neg_vals_ordered
    axes[idx].plot(x, vals, marker='o', linestyle='--', label=symptom + " Negative", color='red')
    cur_max = max(vals)
    cur_min = min(vals)
    max_val = max(max_val, cur_max)
    min_val = min(min_val, cur_min)
    # axes[idx].plot(x, list(map(float, results[symptom]['performance_score1'])), marker='o')
    random_values = []
    for i in range(10):
        cur_vals = list(map(float, results["random "+str(i)]['behaviour_score1']))
        random_values.append([float(zero_val)] + cur_vals)
    random_values = np.array(random_values)
    random_mean = np.mean(random_values, axis=0)
    lower_bound = np.mean(random_values, axis=0) - stats.sem(random_values, axis=0)
    sem = stats.sem(random_values)
    ci = stats.t.ppf(0.95, len(random_values) - 1) * sem
    lower_bound = random_mean - ci
    upper_bound = random_mean + ci
    axes[idx].plot(x, random_mean, marker='o', linestyle='--', label='Random Mean', color='orange')

    axes[idx].fill_between(x, lower_bound, upper_bound, color='orange', alpha=0.3, label='Random ±95% CI')
    # random_values = np.mean(random_values, axis=0)

    axes[idx].set_title(symptom, fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Intensity')
    axes[idx].set_ylabel('Score')

for i in range(len(symptom_names)):
    lower_bound = min_val - abs(min_val) * 0.1
    upper_bound = max_val + abs(max_val) * 0.1
    axes[i].set_ylim(lower_bound, upper_bound)

    
for i in range(n_symptoms, len(axes)):
    fig.delaxes(axes[i])

fig.suptitle("Behaviour score1 on "+ cur_task, fontsize=20, fontweight='bold')
plt.tight_layout()
plt.savefig("figures/" + model + cur_task + "_" + results["random 0"]['behaviour_score1_name'][0] +".png", dpi=600)
plt.close()

In [17]:
fig, axes = plt.subplots(rows, cols, figsize=(15, 3 * rows))
axes = axes.flatten()
# plt.title('Good Scores on SocialChemistry101')
max_val = None
min_val = None
for idx, symptom in enumerate(symptom_names):
    zero_val = results["none"]['behaviour_score2'][0]
    vals = [float(zero_val)]
    vals = vals + list(map(float, results[symptom]['behaviour_score2'][1:]))
    axes[idx].plot(x, vals, marker='o', label=symptom, color='blue')

    cur_max = max(vals)
    cur_min = min(vals)

    if max_val is None:
        max_val = cur_max
    else:
        max_val = max(max_val, cur_max)
    if min_val is None:
        min_val = cur_min
    else:
        min_val = min(min_val, cur_min)

    vals = [float(zero_val)]
    neg_vals = list(map(float, results[symptom]['behaviour_score2'][:1]))
    neg_vals_ordered = []
    for i in range(1):
        neg_vals_ordered.append(neg_vals[0-i])
    vals = vals + neg_vals_ordered
    axes[idx].plot(x, vals, marker='o', linestyle='--', label=symptom + " Negative", color='red')

    cur_max = max(vals)
    cur_min = min(vals)
    max_val = max(max_val, cur_max)
    min_val = min(min_val, cur_min)
        
    # axes[idx].plot(x, list(map(float, results[symptom]['performance_score1'])), marker='o')
    random_values = []
    for i in range(10):
        cur_vals = list(map(float, results["random "+str(i)]['behaviour_score2']))
        random_values.append([float(zero_val)] + cur_vals)
    random_values = np.array(random_values)
    random_mean = np.mean(random_values, axis=0)
    lower_bound = np.mean(random_values, axis=0) - stats.sem(random_values, axis=0)
    sem = stats.sem(random_values)
    ci = stats.t.ppf(0.95, len(random_values) - 1) * sem
    lower_bound = random_mean - ci
    upper_bound = random_mean + ci
    axes[idx].plot(x, random_mean, marker='o', linestyle='--', label='Random Mean', color='orange')

    axes[idx].fill_between(x, lower_bound, upper_bound, color='orange', alpha=0.3, label='Random ±95% CI')
    # random_values = np.mean(random_values, axis=0)

    axes[idx].set_title(symptom, fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Intensity')
    axes[idx].set_ylabel('Score')

for i in range(len(symptom_names)):
    lower_bound = min_val - abs(min_val) * 0.1
    upper_bound = max_val + abs(max_val) * 0.1
    axes[i].set_ylim(lower_bound, upper_bound)
    
for i in range(n_symptoms, len(axes)):
    fig.delaxes(axes[i])

fig.suptitle("Behaviour score2 on "+ cur_task, fontsize=20, fontweight='bold')
plt.tight_layout()
plt.savefig("figures/" + model + cur_task + "_" + results["random 0"]['behaviour_score2_name'][0] +".png", dpi=600)
plt.close()